In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from fractions import Fraction
import numpy as np
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\Quantomatic.jar'
zx.tikz.tikzit_location = r'C:\Users\John\Documents\tikzit\tikzit.exe'

In [4]:
def generate_CNOT_circuit(qubits, depth):
    c = zx.Circuit(qubits)
    for _ in range(depth):
        tgt = random.randrange(qubits)
        while True:
            ctrl = random.randrange(qubits)
            if ctrl!=tgt: break
        c.add_gate("CNOT",tgt,ctrl)
    return c

def generate_matrix(qubits, c=None):
    m = zx.Mat2.id(qubits)
    if not c: c = generate_CNOT_circuit(qubits, 2*qubits*qubits)
    for cnot in c.gates:
        m.col_add(cnot.control, cnot.target)
    return m

In [9]:
c = zx.Circuit(3)
c.add_gate("CNOT",0,1)
c.add_gate("CNOT",2,1)
m = generate_matrix(3,c)
m

[ 1  1  0 ]
[ 0  1  0 ]
[ 0  1  1 ]

In [10]:
m.to_cnots()

[CNOT(2,1), CNOT(0,1)]

In [14]:
#random.seed(1337)
c = generate_CNOT_circuit(6, 50)
m = generate_matrix(6, c)
#c2 = zx.Circuit(6)
#c2.gates = list(m.to_cnots())
#zx.compare_tensors(c,c2,False)
print(m)

[ 0  1  1  1  0  1 ]
[ 0  1  1  1  0  0 ]
[ 1  1  1  0  1  1 ]
[ 1  1  1  1  0  1 ]
[ 1  0  0  0  1  1 ]
[ 0  1  0  1  0  0 ]


In [35]:
def random_matrix(r,c):
    data = [[random.randint(0,1) for i in range(c)] for j in range(r)]
    return zx.Mat2(data)

#print(random_matrix(5,8))

[ 0  1  1  1  0  1  1  0 ]
[ 1  1  1  1  1  1  1  1 ]
[ 0  1  0  0  1  0  1  1 ]
[ 0  1  0  1  1  0  0  0 ]
[ 1  0  0  1  1  1  0  1 ]


In [30]:
from pyzx.circuit.gates import CNOT
cnots = [CNOT(3,0), CNOT(7,4), CNOT(28,27), CNOT(27,0), CNOT(2,1), CNOT(3,27), CNOT(27,3), CNOT(24,6), CNOT(25,6), CNOT(29,6), CNOT(7,6), CNOT(10,8), CNOT(18,17), CNOT(14,12), CNOT(15,14), CNOT(17,16), CNOT(21,20), CNOT(22,20), CNOT(22,21), CNOT(26,24), CNOT(25,26), CNOT(26,25), CNOT(26,29), CNOT(29,26), CNOT(29,28), CNOT(8,9), CNOT(13,14), CNOT(12,14), CNOT(10,11), CNOT(7,29), CNOT(4,6), CNOT(3,29), CNOT(24,29), CNOT(24,28), CNOT(25,26), CNOT(6,25), CNOT(24,25), CNOT(0,24), CNOT(3,24), CNOT(22,23), CNOT(12,14), CNOT(13,14), CNOT(12,13), CNOT(5,6), CNOT(3,6), CNOT(10,11), CNOT(8,9), CNOT(0,3), CNOT(1,3), CNOT(0,2), CNOT(0,1)]

In [62]:
while True:
    #c = generate_CNOT_circuit(30, 50)
    #m = generate_matrix(30, c)
    m = random_matrix(7,10)
    cnots = m.to_cnots(optimize=True)
    for i in range(len(cnots)):
        for j in range(i+1,len(cnots)):
            if cnots[i] == cnots[j]:
                #if True:
                if not any(cnot.control == cnots[i].target for cnot in cnots[i:j]):
                    print(cnots[i])
                    raise Exception
cnots

CNOT(4,6)


Exception: 

In [47]:
m

[ 1  0  1  1 ]
[ 1  0  1  1 ]
[ 1  1  0  0 ]

In [48]:
m2 = m.copy()
for cnot in cnots:
    m2.row_add(cnot.target,cnot.control)
    print(m2)
    print()

[ 1  0  1  1 ]
[ 0  0  0  0 ]
[ 1  1  0  0 ]

[ 1  0  1  1 ]
[ 0  0  0  0 ]
[ 0  1  1  1 ]

[ 1  0  1  1 ]
[ 0  1  1  1 ]
[ 0  1  1  1 ]

[ 1  0  1  1 ]
[ 0  1  1  1 ]
[ 0  0  0  0 ]

[ 1  1  0  0 ]
[ 0  1  1  1 ]
[ 0  0  0  0 ]

[ 1  0  1  1 ]
[ 0  1  1  1 ]
[ 0  0  0  0 ]



In [67]:
def filter_duplicate_cnots(cnots):
    qubits = max([max(cnot.control,cnot.target) for cnot in cnots]) + 1
    c = zx.Circuit(qubits)
    c.gates = cnots.copy()
    c = zx.optimize.basic_optimization(c,do_swaps=False)
    return c.gates

filter_duplicate_cnots(cnots)

[CNOT(2,1),
 CNOT(4,1),
 CNOT(5,1),
 CNOT(6,3),
 CNOT(6,2),
 CNOT(4,6),
 CNOT(3,0),
 CNOT(3,1),
 CNOT(4,3),
 CNOT(5,3),
 CNOT(6,4),
 CNOT(1,2),
 CNOT(3,2),
 CNOT(0,4),
 CNOT(1,6),
 CNOT(2,3)]

In [66]:
cnots

[CNOT(2,1),
 CNOT(4,1),
 CNOT(5,1),
 CNOT(6,3),
 CNOT(3,0),
 CNOT(3,1),
 CNOT(4,3),
 CNOT(5,3),
 CNOT(6,2),
 CNOT(3,2),
 CNOT(4,6),
 CNOT(6,4),
 CNOT(4,6),
 CNOT(0,2),
 CNOT(4,6),
 CNOT(1,6),
 CNOT(0,4),
 CNOT(1,2),
 CNOT(0,2),
 CNOT(2,3)]

In [5]:
len(m.to_cnots())

17

In [6]:
s = zx.linalg.column_optimal_swap(m)
s

{2: 1, 4: 4, 5: 0, 1: 3, 3: 5, 0: 2}

In [31]:
data = []
for row in m.data:
    r = [row[s[i]] for i in range(len(row))]
    data.append(r)
print(data)
m2 = zx.Mat2(data)

[[0, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 0], [0, 1, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1]]


In [32]:
m2

[ 0  0  1  1  0  0 ]
[ 1  1  0  1  1  0 ]
[ 0  1  1  1  0  1 ]
[ 0  1  1  0  0  0 ]
[ 0  0  0  1  1  1 ]
[ 0  1  0  1  0  1 ]

In [33]:
len(m2.to_cnots())

15

In [7]:
def gauss_depth(m, full_reduce=False, x=None):
    rows = m.rows()
    for i in range(rows):
        available = list(range(i,rows))
        if m.data[i][i] == 0:
            for j in range(i+1,rows):
                if m.data[j][i] == 1:
                    m.row_add(j,i)
                    if x: x.row_add(j,i)
                    available.remove(i)
                    available.remove(j)
                    break
        if not available: available = list(range(i,rows))
        while available:
            c1 = available.pop(0)
            if not m.data[c1][i]: 
                if not available:
                    if any(m.data[j][i] for j in range(i+1,rows)):
                        available = list(range(i,rows))
                continue
            while len(available):
                c2 = available.pop(0)
                if not m.data[c2][i]: continue
                m.row_add(c1,c2)
                if x: x.row_add(c1,c2)
                break
            else: # Didn't find suitable other one
                if c1 == i: # Column is completely cleared out
                    break
                else:
                    m.row_add(i,c1)
                    if x: x.row_add(i,c1)
                    available = list(range(i+1,rows))
                    continue
            if not available:
                if any(m.data[j][i] for j in range(i+1,rows)):
                    available = list(range(i,rows))
        
    if not full_reduce: return
    for i in range(rows-1,-1,-1):
        available = list(range(i,-1,-1))
        while available:
            c1 = available.pop(0)
            if not m.data[c1][i]:
                if not available:
                    if any(m.data[j][i] for j in range(i)):
                        available = list(range(i,-1,-1))
                continue
            while len(available):
                c2 = available.pop(0)
                if not m.data[c2][i]: continue
                m.row_add(c1,c2)
                if x: x.row_add(c1,c2)
                break
            else: # Didn't find suitable other one
                if c1 == i: # Column is completely cleared out
                    break
                else:
                    m.row_add(i,c1)
                    if x: x.row_add(i,c1)
                    available = list(range(i-1,-1,-1))
                    continue
            if not available:
                if any(m.data[j][i] for j in range(i)):
                    available = list(range(i,-1,-1))

class Printer(object):
    def row_add(self,r1,r2):
        print(r1,r2)

from pyzx.circuit.gates import CNOT
class CNOTMaker:
    def __init__(self):
        self.cnots = []
    def row_add(self, r1, r2):
        self.cnots.append(CNOT(r2,r1))

def to_cnots_depth(m):
    cn = CNOTMaker()
    gauss_depth(m.copy(), full_reduce=True, x=cn)
    return cn.cnots

In [8]:
def basic_gauss(m, full_reduce=False, x=None):
    rows = m.rows()
    #First do all the pivots
    for i in range(rows):
        if m.data[i][i] == 0:
            for j in range(i+1, rows):
                if m.data[j][i] == 1:
                    m.row_add(j,i)
                    if x: x.row_add(j,i)
                    break
        for j in range(i+1,rows):
            if m.data[j][i]:
                m.row_add(i,j)
                if x: x.row_add(i,j)
    if full_reduce:
        for i in range(rows-1,-1,-1):
            for j in range(i-1,-1,-1):
                if m.data[j][i]:
                    m.row_add(i,j)
                    if x: x.row_add(i,j)

def to_cnots_simple(m):
    cn = CNOTMaker()
    basic_gauss(m.copy(), full_reduce=True, x=cn)
    return cn.cnots

In [12]:
m = generate_matrix(8)
print(m)
#m2 = m.copy()
#basic_gauss(m2,full_reduce=False)
#print()
#print(m2)
c = depth_optimal(m)
c2 = zx.Circuit(m.rows())
c2.gates = m.to_cnots(optimize=True)
zx.compare_tensors(c,c2,False)
zx.d3.draw(c, scale=30)
zx.d3.draw(c2, scale=30)

[ 0  0  0  0  0  1  1  1 ]
[ 1  1  1  0  1  0  1  0 ]
[ 1  1  0  1  0  1  0  1 ]
[ 1  0  0  0  0  1  1  1 ]
[ 0  0  0  0  0  0  1  1 ]
[ 0  0  1  0  0  0  1  0 ]
[ 0  0  1  1  0  1  1  1 ]
[ 0  0  1  0  1  0  0  0 ]


In [8]:
g = c.to_graph(compress_rows=False)
zx.d3.draw(g,scale=30)
g2 = c2.to_graph(compress_rows=False)
zx.d3.draw(g2,scale=30)

In [10]:
def reduce_cnot_depth(c):
    q = c.qubits
    cnots = c.gates
    pivots = []
    #if c.gates[0].control == 0: # We have pivoted onto first row
    for i, g in enumerate(cnots):
        if g.control < g.target: pivots.append(i)
    
    added = 0
    for k, i in enumerate(pivots):
        cnot = cnots.pop(i+added)
        for j in range(i+added-1,k-1,-1):
            g = cnots[j]
            if g.control == cnot.target:
                cnots.insert(j, CNOT(cnot.control, g.target))
                added += 1
            elif g.target == cnot.control: 
                cnots.insert(j, CNOT(g.control, cnot.target))
                added += 1
        cnots.insert(k,cnot) 

def depth_optimal(m):
    cn = CNOTMaker()
    m2 = m.copy()
    basic_gauss(m2, full_reduce=False, x=cn)
    c = zx.Circuit(m.rows())
    c.gates = cn.cnots
    #zx.d3.draw(c)
    reduce_cnot_depth(c)
    cn2 = CNOTMaker()
    basic_gauss(m2, full_reduce=True, x=cn2)
    c.gates.extend(cn2.cnots)
    c = zx.optimize.basic_optimization(c,do_swaps=False)
    return c

In [ ]:
random.seed(42)
xs = [3*i for i in range(1,20)]
yys = [[],[]]
zzs = [[],[]]

reps = 40

for qubits in xs:
    print(qubits, end='; ')
    depth = qubits**2*2
    depth1 = 0
    depth2 = 0
    count1 = 0
    count2 = 0
    for _ in range(reps):
        c_orig = generate_CNOT_circuit(qubits, depth)
        m = generate_matrix(qubits, c_orig)
        c = depth_optimal(m)
        c2 = zx.Circuit(qubits)
        c2.gates = m.to_cnots(optimize=True)
        depth1 += c.to_graph().depth()-2
        depth2 += c2.to_graph().depth()-2
        count1 += len(c.gates)
        count2 += len(c2.gates)
    
    yys[0].append(depth1/reps)
    yys[1].append(depth2/reps)
    zzs[0].append(count1/reps)
    zzs[1].append(count2/reps)
    
    #print(depth1/reps, depth2/reps)
    #print(count1/reps, count2/reps)

3; 6; 9; 12; 15; 18; 21; 24; 27; 30; 33; 36; 39; 42; 45; 48; 51; 54; 57; 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
colours = ['#53257f', '#bc1b73', '#f8534a', '#ffa600']
names = ['depth-optimal','other']
styles = ['-','--','-.',':']

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
for i, ys in enumerate(yys):
    ax1.plot(xs, ys, c=colours[i], marker="o",markersize=3, linestyle=styles[i], label=names[i])

ax1.set_ylabel("depth")
ax1.set_xlabel("qubits")
plt.legend(loc='lower right');
plt.grid(color='#EEEEEE')
plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
for i, zs in enumerate(zzs):
    ax1.plot(xs, zs, c=colours[i], marker="o",markersize=3, linestyle=styles[i], label=names[i])

ax1.set_ylabel("count")
ax1.set_xlabel("qubits")
plt.legend(loc='lower right');
plt.grid(color='#EEEEEE')
plt.show()

In [14]:
c = zx.Circuit.load(r'..\circuits\ibmq\9symml_195.qasm')
print(c.stats())

Circuit 9symml_195.qasm on 16 qubits with 34881 gates.
        15232 is the T-count
        19649 Cliffords among which 
        15232 2-qubit gates and 4352 Hadamard gates.


In [12]:
zx.d3.draw(c)

In [16]:
g = c.to_graph()
g2 = zx.teleport_reduce(g)
c2 = zx.Circuit.from_graph(g2).to_basic_gates()
print(c2.to_basic_gates().stats())
c3 = zx.optimize.basic_optimization(c2).to_basic_gates()
print(c3.stats())
#zx.d3.draw(c4)

Circuit  on 16 qubits with 30364 gates.
        6450 is the T-count
        23914 Cliffords among which 
        15232 2-qubit gates and 4352 Hadamard gates.
Circuit  on 16 qubits with 26805 gates.
        6450 is the T-count
        20355 Cliffords among which 
        13746 2-qubit gates and 2380 Hadamard gates.


In [17]:
c4 = zx.optimize.full_optimize(c3.to_basic_gates(),quiet=False).to_basic_gates()
print(c4.stats())

1.new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new bloc

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block


new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block

new block
